In [ ]:
#####  台彩大樂透

import requests
from bs4 import BeautifulSoup as bs

url = "http://www.taiwanlottery.com.tw/Lotto/Lotto649/history.aspx"

headers = {
    "User-Agent": "Mozilla/5.0 (Macintosh; Intel Mac OS X 10_12_6) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/90.0.4430.212 Safari/537.36"
}

### 發 POST requests 取得頁面 source code
payload = {
    "__VIEWSTATE": "",
    "__VIEWSTATEGENERATOR": "",
    "__EVENTVALIDATION": "",
    "forma": "請選擇遊戲",
    "Lotto649Control_history$txtNO": "", # 配入空字串
    "Lotto649Control_history$chk": "radYM",
    "Lotto649Control_history$dropYear": 108,
    "Lotto649Control_history$dropMonth": 3,
    "Lotto649Control_history$btnSubmit": "查詢"
}

res = requests.post(url,headers=headers,data=payload)  ### data 為要攜帶的參數
# print(res.text)
soup = bs(res.text,"lxml")
# print(soup)

In [ ]:
### 整理解析資料的 function 
def get_clear_data(soup):
    tables = soup.select("table#Lotto649Control_history_dlQuery table")
    
    result = []
    
    for table_tag in tables:

        # 期別 , 開獎日 -> 第二個 tr
        tr2 = table_tag.select("tr")[1]

        lotto_no   = tr2.select("td")[0].text
        lotto_date = tr2.select("td")[1].text.strip()

        # 樂透號碼 -> 第五個 tr
        tr5 = table_tag.select("tr")[4]

        # List comprehension
        lotto_numbers_short = [ td_tag.text.strip() for td_tag in tr5.select("td")[1:] ]
        
        result.append({
            "no"   : lotto_no,
            "time" : lotto_date,
            "numbers" : lotto_numbers_short
        })

    return result

# get_clear_data(soup)

In [ ]:
### 取得隱藏參數
def get_hidden_params(soup):
    __VIEWSTATE = soup.select("input#__VIEWSTATE")[0]["value"]
    __VIEWSTATEGENERATOR = soup.select("input#__VIEWSTATEGENERATOR")[0]["value"]
    __EVENTVALIDATION = soup.select("input#__EVENTVALIDATION")[0]["value"]
    
    return {
        "__VIEWSTATE" : __VIEWSTATE,
        "__VIEWSTATEGENERATOR" : __VIEWSTATEGENERATOR,
        "__EVENTVALIDATION" : __EVENTVALIDATION
    }


# get_hidden_params(soup)

In [ ]:
### 抓取 1 ~ 5 月 樂透號碼

url = "http://www.taiwanlottery.com.tw/Lotto/Lotto649/history.aspx"

headers = {
    "User-Agent": "Mozilla/5.0 (Macintosh; Intel Mac OS X 10_12_6) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/90.0.4430.212 Safari/537.36"
}

### 發 POST requests 取得頁面 source code
for i in range(1,6):
    hidden = get_hidden_params(soup)
    payload = {
        "__VIEWSTATE": hidden["__VIEWSTATE"],
        "__VIEWSTATEGENERATOR": hidden["__VIEWSTATEGENERATOR"],
        "__EVENTVALIDATION": hidden["__EVENTVALIDATION"],
        "forma": "請選擇遊戲",
        "Lotto649Control_history$txtNO": "", # 配入空字串
        "Lotto649Control_history$chk": "radYM",
        "Lotto649Control_history$dropYear": 108,
        "Lotto649Control_history$dropMonth": i,
        "Lotto649Control_history$btnSubmit": "查詢"
    }

    res = requests.post(url,headers=headers,data=payload)  ### data 為要攜帶的參數
    soup = bs(res.text,"lxml")
    
    data = get_clear_data(soup)
    print(data)
    print("-"*80)